In [1]:
import os
os.chdir("/home/puneetj/Data_DL")
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
from shutil import copyfile
from vgg_bn_with_aug import Vgg16BN
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint

Using Theano backend.


In [2]:
data_path = "/home/puneetj/Cats_Dogs" + '/' 
train_path = data_path + 'train/'
valid_path = data_path + 'validation/'
test_path =  data_path + 'test/'
model_path = data_path + 'models/'
submission_path = data_path + 'submissions/'

# data
img_width, img_height = 224, 224
batch_size = 1024
nb_train_samples = 20000
nb_valid_samples = 5000
nb_test_samples = 12500
classes = ["cats", "dogs"]
n_classes = len(classes)

# model
nb_epoch = 10
nb_aug = 1
lr = 1

In [3]:
vgg = Vgg16BN(size=(img_width, img_height), n_classes=n_classes, batch_size=batch_size, lr=lr,weights_file =
            '/home/puneetj/Cats_Dogs/models/0.10-loss_224x224_10epoch_5aug_0.001lr_vgg16-bn.h5')
model = vgg.model
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

/home/puneetj/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:622: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [ ]:
# generate predictions
for aug in range(nb_aug):
    print("Generating predictions for Augmentation {0}...",format(aug+1))
    if aug == 0:
        predictions, filenames = vgg.test(test_path, nb_test_samples, aug=nb_aug)
    else:
        aug_pred, filenames = vgg.test(test_path, nb_test_samples, aug=nb_aug)
        predictions += aug_pred

print("Averaging Predictions Across Augmentations...")

('Generating predictions for Augmentation {0}...', '1')
Found 12500 images belonging to 1 classes.

In [10]:
predictions /= nb_aug

In [12]:
sub_file = submission_path + info_string + '9.csv'

with open(sub_file, 'w') as f:
    print("Writing Predictions to CSV...")
    f.write('id,label\n')
    for i, image_name in enumerate(filenames):
        pred = ['%.6f' % p for p in predictions[i, :]]
        if i % 2500 == 0:
            print(i, '/', nb_test_samples)
        f.write('%s,%s\n' % (os.path.basename(image_name).replace('.jpg', ''), (pred[1])))
    print("Done.")

Writing Predictions to CSV...
(0, '/', 12500)
(2500, '/', 12500)
(5000, '/', 12500)
(7500, '/', 12500)
(10000, '/', 12500)
Done.
